In [7]:
import pandas as pd
from tqdm.contrib.concurrent import thread_map

c:\Users\PC\anaconda3\envs\smart_menu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create dictionary

In [8]:
df = pd.read_excel('../data_sample/Data_Labeling.xlsx')

df.head()

,ImageName,VietnameseName,EnglishName,Price
0,001.jpeg,COMBO 1,COMBO 1,169000
1,001.jpeg,COMBO 2,COMBO 2,169000
2,001.jpeg,COMBO 3,COMBO 3,169000
3,001.jpeg,RƯỢU SOJU,SOJU,NOT GIVEN
4,001.jpeg,RƯỢU VODKA,VODKA,NOT GIVEN


In [9]:
df.tail()

,ImageName,VietnameseName,EnglishName,Price
15206,850.jpeg,SÒ ĐIỆP NƯỚNG,GRILLED SCALLOP,10000
15207,850.jpeg,KHÔ MỰC,DRIED SQUID,NOT GIVEN
15208,850.jpeg,CÁ SƠN NƯỚNG,GRILLED CARDINAL FISH,NOT GIVEN
15209,850.jpeg,CÁ CHỈ VÀNG,YELLOWSTRIPE SCAD,60000
15210,850.jpeg,CÁ LAO NƯỚNG,GRILLED RED CORNETFISH,99000


In [10]:
#get Vietnamese food names
food_names = df['VietnameseName'].values
type(food_names)

numpy.ndarray

In [87]:
#Clean text, remove punification number and etc
import re

def clean_food_name(food_name):
    #Remove quantity for example 500ML 1L 20KG ....
    food_name = food_name.lower()
    food_name = re.sub(r'\d+\w+', ' ', food_name)
    #Remove purification and digits
    food_name = re.sub(r"[^\w\s]|\d", ' ', food_name)
    #Remove size entites
    token = r"\s+x{0,2}[xlms][\.:-]?\b|(nhỏ|vừa|lớn|bự|to|small|medium|big|large):?"
    food_name = re.sub(token, " ", food_name)
    #Remove specail token
    food_name = re.sub(r"_x", ' ', food_name)
    
    food_name = food_name.strip().split()
    return food_name
    

test = clean_food_name('BELGIAN DARK 8.1%/IBU 32 -    X 500ML (PHỞ ĐÙI + TRỨNG NON) X L M nhỏ vừa_x hải sản xuân')
test

['belgian', 'dark', 'ibu', 'phở', 'đùi', 'trứng', 'non', 'hải', 'sản', 'xuân']

In [88]:
def get_big_gram(text, n=2, m=2):
    words = clean_food_name(text)
    big_grams = []
    
    for k in range(n,m+1):
        for i in range(len(words)):
            big_gram = ''
            if i + k > len(words):
                continue
            
            for j in range(k):
                big_gram += words[i+j] + ' '
                
            big_grams.append(big_gram.strip())
            
    return big_grams

print(get_big_gram('BELGIAN DARK 8.1%/IBU 32 - 500ML (PHỞ ĐÙI + TRỨNG NON) hải sản'))
                

['belgian dark', 'dark ibu', 'ibu phở', 'phở đùi', 'đùi trứng', 'trứng non', 'non hải', 'hải sản']


In [89]:
food_vocabulary = thread_map(clean_food_name, food_names, max_workers=6)
food_vocabulary_big_grams = thread_map(get_big_gram, food_names, max_workers=6)


100%|██████████| 15211/15211 [00:00<00:00, 294910.94it/s]


In [90]:
food_vocabulary = [item for sublist in food_vocabulary for item in sublist]
print(food_vocabulary[:10])

['combo', 'combo', 'combo', 'rượu', 'soju', 'rượu', 'vodka', 'tiger', 'lon', 'tiger']


In [91]:
food_vocabulary_big_grams = [item for sublist in food_vocabulary_big_grams for item in sublist]
print(food_vocabulary_big_grams[:20])

['rượu soju', 'rượu vodka', 'tiger lon', 'tiger chai', 'tiger bạc', 'bạc chai', 'tiger bạc', 'bạc lon', 'bò húc', 'bia quy', 'quy nhơn', 'bia bivina', 'strong bow', 'rau muống', 'muống xào', 'xào tỏi', 'mồng tơi', 'tơi xào', 'xào tỏi', 'cải xào']


In [92]:
from collections import Counter

corpus = Counter(food_vocabulary)
corpus.most_common(10)

[('cá', 1513),
 ('trà', 1312),
 ('nướng', 1174),
 ('sữa', 1117),
 ('chiên', 1053),
 ('bò', 1014),
 ('xào', 823),
 ('gà', 779),
 ('tôm', 710),
 ('cơm', 620)]

In [93]:
corpus_big_grams = Counter(food_vocabulary_big_grams)
corpus_big_grams.most_common(10)

[('trà sữa', 472),
 ('hải sản', 374),
 ('phô mai', 322),
 ('trân châu', 252),
 ('cơm chiên', 245),
 ('cá hồi', 240),
 ('sữa chua', 212),
 ('chiên mắm', 177),
 ('thập cẩm', 166),
 ('muối ớt', 161)]

In [94]:
#Save corpus
with open('food_vocabulary.txt', 'w', encoding='utf-8') as f:
    for food, count in corpus.most_common():
        if count > 1:
            save_format = f"{food}${count}\n"
            f.write(save_format)

with open('food_vocabulary_big_grams.txt', 'w', encoding='utf-8') as f:
    for food, count in corpus_big_grams.most_common():
        if count > 1:
            save_format = f"{food}${count}\n"
            f.write(save_format)


# Spell correct with symspell

In [95]:
import symspellpy
import random

import string

## Generate test cases

In [96]:
def random_delete(text, percent = 0.1):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = random.randint(0, len(text_aggumented)-1)
        del text_aggumented[k]
        
    return "".join(text_aggumented)

def random_replace(text, percent = 0.2):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = -1
        while k == -1 or text[k] == ' ':
            k = random.randint(0, len(text)-1)

        char = random.choice(string.ascii_uppercase)
        text_aggumented[k] = char
        
    return "".join(text_aggumented)

def random_swap(text, percent = 0.1):
    n = int(percent*len(text))+1
    text_aggumented = list(text)
    
    for i in range(n):
        k = -1
        while k == -1 or text[k] == ' ':
            k = random.randint(0, len(text)-1)
        h = k
        while h == k or text[h] == ' ':
            h = random.randint(0, len(text)-1)
        text_aggumented[k], text_aggumented[h] = text_aggumented[h], text_aggumented[k]
        
    return "".join(text_aggumented)

def lower(text):
    text = text.lower()
    return text

def random_text_aggument(text):
    k = random.randint(0, 2)
    random_agg = {0: random_delete, 1:random_replace, 2:random_swap}
    return random_agg[k](text, percent=0.125)
    
random_text_aggument('BELGIAN DARK 8.1%IBU 32 - 500ML (PHỞ ĐÙI + TRỨNG NON)')
    

'BELGIAN DARK 8.1%IB32 - 50L (PHỞ ÙI+ TRỨN NON)'

In [97]:
TEST_CASE = random.choices(food_names, k = 500)
TEST_CASE = thread_map(lower, TEST_CASE, max_workers=6)
TEST_CASE_WRONG = thread_map(random_text_aggument, TEST_CASE, max_workers=6)

print(TEST_CASE[:7])
print(TEST_CASE_WRONG[:7])

100%|██████████| 500/500 [00:00<00:00, 249869.18it/s]

['cá kèo nấu rau răm', 'bánh canh', 'pizza khoai tây size m', 'nem chua chiên hà nội', 'trà lipton', 'lẩu gà lá giang', 'đậu bắp luộc']
['rr ăèo nấu cau ákm', 'bánh ch', 'pizza kRoai QAy size m', 'nim chue chaêộ hà nni', 'trà MPpton', 'lẩC gà lá gianG', 'đậu bắp lộ']


## METRIC

In [98]:
def cer(pred, true):
    n = len(true)
    wrong = 0
    for c1, c2 in zip(pred, true):
        if c1 != c2:
            wrong += 1
    
    
    return (n - wrong)/n

def wer(pred, true):
    pred = pred.split()
    true = true.split()
    n = len(true)
    wrong = 0
    for c1, c2 in zip(pred, true):
        if c1 != c2:
            wrong += 1
    
    return (n - wrong)/n

print(cer("tomorrow now today and tomorrow", "tomorrow now today and tomoraow"))
wer("tomorrow now today and tomorrow", "tomorrow now today and tomoraow")


0.967741935483871


0.8

## Spell check

In [99]:
from symspellpy import SymSpell, Verbosity
from itertools import islice
import numpy as np

EDIT_DISTANCE = 3

spell_check = SymSpell(max_dictionary_edit_distance=EDIT_DISTANCE)
spell_check.load_dictionary('food_vocabulary.txt', 0, 1, 
                                    encoding='utf-8', separator='$')
spell_check.load_bigram_dictionary('food_vocabulary_big_grams.txt', 0, 1, 
                                    encoding='utf-8', separator='$')

print(list(islice(spell_check.words.items(), 5)))
print(list(islice(spell_check.bigrams.items(), 5)))


[('cá', 1513), ('trà', 1312), ('nướng', 1174), ('sữa', 1117), ('chiên', 1053)]
[('trà sữa', 472), ('hải sản', 374), ('phô mai', 322), ('trân châu', 252), ('cơm chiên', 245)]


In [100]:
def correct_spell(text):
    import re 
    
    text = re.sub(r'[.\?#@+,<>%~`!$^&\(\):;\\\/]', r' \g<0> ', text)
    
    suggestion = spell_check.lookup_compound(text, max_edit_distance=EDIT_DISTANCE,
                                             ignore_non_words=True, ignore_term_with_digits=True)
    
    return suggestion[0]._term

In [101]:
%%time
test_result = []
wer_result = []
cer_result = []
N = 7
for test_case, val in zip(TEST_CASE_WRONG, TEST_CASE):
    correct_text = correct_spell(test_case)
    
    test_result.append(correct_text)
        
        
    wer_result.append(
        wer(correct_text, val)
    )
    
    cer_result.append(
        cer(correct_text, val)
    )
    

print("Example: ")
print(TEST_CASE[:N])
print(TEST_CASE_WRONG[:N])
print(test_result[:N])

print("Metric:", f"WER: {np.mean(wer_result):.3f}", f"CER: {np.mean(cer_result):.3f}")



Example: 
['cá kèo nấu rau răm', 'bánh canh', 'pizza khoai tây size m', 'nem chua chiên hà nội', 'trà lipton', 'lẩu gà lá giang', 'đậu bắp luộc']
['rr ăèo nấu cau ákm', 'bánh ch', 'pizza kRoai QAy size m', 'nim chue chaêộ hà nni', 'trà MPpton', 'lẩC gà lá gianG', 'đậu bắp lộ']
['ri xèo nấu cau tôm', 'bánh cá', 'pizza khoai cay size m', 'nấm chua chiên hà nội', 'trà m non', 'lẩu gà lá giang', 'đậu bắp lá']
Metric: WER: 0.689 CER: 0.731
CPU times: total: 469 ms
Wall time: 1.03 s
